In [1]:
!pip install pydot
!pip install pydotplus
!pip install graphviz

In [1]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from tkinter import filedialog

import tensorflow as tf
from tensorflow import keras
import cv2

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

In [3]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [26]:
def addaptive_histogram(img,clahe):
        H_list = [];
        if len(img.shape) == 3:
            r, g, b = cv2.split(img)
            lit = [r, g, b]
            for img1 in lit:          
                equ = clahe.apply(img1)
                H_list.append(equ)
            H_img = cv2.merge((H_list[0], H_list[1], H_list[2]))
        else:
            H_img = clahe.apply(img)
            
        return H_img

In [27]:
# Insertion over union ( for merging overlapped segmented )
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [28]:
def U_net(modelPath, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
    # Build U-Net model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255) (inputs)
    
    c1 = Conv2D(16, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (s)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(16, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)
    
    c2 = Conv2D(32, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)
    
    c3 = Conv2D(64, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(64, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)
    
    c4 = Conv2D(128, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (p3)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (c4)
    p4 = MaxPooling2D(pool_size = (2, 2)) (c4)
    
    c5 = Conv2D(256, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (p4)
    c5 = Dropout(0.3) (c5)
    c5 = Conv2D(256, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (c5)
    
    u6 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation = 'elu', kernel_initializer='he_normal', padding = 'same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation = 'elu', kernel_initializer='he_normal', padding = 'same') (c6)
    
    u7 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (c7)
    
    u8 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (c8)
    
    u9 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same') (c8)
    u9 = concatenate([u9, c1], axis = 3)
    c9 = Conv2D(16, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation = 'elu', kernel_initializer = 'he_normal', padding = 'same') (c9)
    
    outputs = Conv2D(1, (1, 1), activation = 'sigmoid') (c9)
    
    model = Model(inputs = [inputs], outputs = [outputs])
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [mean_iou])
    model.load_weights(modelPath)
    #model.summary()
    return model

In [29]:
def prediction(model, imagePath, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
    img=cv2.imread(imagePath,0)
    maskl = cv2.threshold(img, 190,255, cv2.THRESH_BINARY)
    img=np.expand_dims(img,axis = -1)
    x_test= np.zeros((1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype = np.uint8)
    #testimg = resize(img,(self.IMG_HEIGHT,self.IMG_WIDTH),mode = 'constant',preserve_range = True)
    x_test[0] = img
    preds_test = model.predict(x_test, verbose = 1)
    #print(preds_test)
    preds_test = ((preds_test >= 0.5))
    mask1 = np.uint8(preds_test[0])
    mask1[mask1 == 1] = 255
    #cv2.imshow('', maskl[1])
    #cv2.waitKey(0)
    return x_test[0], maskl[1]

In [30]:
IMG_HEIGHT = 512
IMG_WIDTH = 512
IMG_CHANNELS = 3
modelPath = 'model-dsbowl2018-1.h5'

trained_model = U_net(modelPath, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

In [31]:
# ------------------ errors --------------------------
# plot_model(trained_model, show_shapes = True)
# loss = trained_model.history['loss']
# val_loss = trained_model.history['val_loss']

In [32]:
clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (8,8))
# Select Image
# filename = filedialog.askopenfilename(title = 'open')
# Read Image From Directory
img1 = cv2.imread("hurricane-florence_00000019_pre_disaster.png")
H_img1 = addaptive_histogram(img1, clahe)
test_image_name1 = "pre_disaster.png"
H_img1 = cv2.resize(H_img1, (512, 512))
cv2.imwrite(test_image_name1, H_img1)

True

In [33]:
# Select Image
# filename = filedialog.askopenfilename(title='open')
# Read Image From Directory
img2 = cv2.imread("hurricane-florence_00000019_post_disaster.png")
H_img2 = addaptive_histogram(img2,clahe)
test_image_name2 = "post_disaster.png"
H_img2 = cv2.resize(H_img2,(512,512))
cv2.imwrite(test_image_name2, H_img2)

True

In [34]:
testimg, mask1 = prediction(trained_model, test_image_name1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

testimg, mask2 = prediction(trained_model, test_image_name2, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

1/1 [==============================] - 0s 80ms/step


In [20]:
Input_image = np.concatenate((cv2.resize(img1,(512, 512)), cv2.resize(img2,(512, 512))), axis = 1)
Enhance_Image = np.concatenate((H_img1, H_img2), axis=1)
segmented_Image = np.concatenate((cv2.resize(mask1,(512, 512)), cv2.resize(mask2,(512, 512))), axis = 1)
mask1 = np.array(mask1)
mask2 = np.array(mask2)

bw_image = np.uint8(((mask1-mask2)))

In [21]:
mark_damage_region = cv2.merge((bw_image, bw_image, bw_image))

plt.imshow(Input_image)
plt.show()

c:\users\kitty\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  after removing the cwd from sys.path.


In [22]:
mark_damage_region = cv2.merge((bw_image, bw_image, bw_image))
#Visualization
cv2.imshow('Input Image', Input_image)
cv2.waitKey(0)
cv2.imshow('Pre_processed Image', Enhance_Image)
cv2.waitKey(0)
cv2.imshow('Segmented Image', segmented_Image)
cv2.waitKey(0)

cv2.imshow('Dmaged Region', mark_damage_region)
cv2.waitKey(0)
#cv2.destroyAllWindows()

32

In [23]:
bw_image[bw_image >= 1] = 1
bw_image = bw_image.flatten();
Total_change= np.sum(bw_image)

In [24]:
mask1=mask1.flatten()
mask1[mask1 >=1 ] = 1
Pre_unchangedpixel = np.sum(mask1)

In [25]:
percentage_damage = (Total_change/(Pre_unchangedpixel))*100
print("Total Percent of Damage in segmented Region ", percentage_damage,' %')

#percentage_damage = (Total_change/(512*512))*100
#print("Total Percent of Damage in completed Image ", percentage_damage,' %')

cv2.imwrite("resized_mask.png",bw_image)

Total Percent of Damage in segmented Region  70.65513703649421  %
Total Percent of Damage in completed Image  9.1949462890625  %


True